## Capstone Project
-------

### Stage 2 - Modelling phase - Neural Networks
------

#### Importing packages and data
------

Possible packages that need to be installed:

1. Hyperas

<code> conda install -c jaikumarm hyperas </code>

These packages are from the previous notebook. If virtual environments are used for neural network, however, the following packages will need to be installed in order for the notebook to run properly.

2. SpaCy

<code> conda install -c spacy spacy </code>

3. 'en_core_web_md' - library used in SpaCy

<code> python -m spacy download en_core_web_md </code>

4. wordcloud

<code> conda install -c conda-forge wordcloud </code>

------

In [1]:
# import packages

# Hyperas/TensorFlow
# the __future__ import command must be in the beginning of the notebook
from __future__ import print_function

from hyperopt import Trials, STATUS_OK, tpe
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform

import tensorflow as tf
from tensorflow.keras.layers import LSTM, BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

# Basics
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np

# Filter warnings
import warnings
warnings.filterwarnings("ignore")

# Preprocessing; model selection and evaluation
from sklearn.model_selection import train_test_split

# text handling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# for custom countvectorizer with SpaCy lemmatization
import spacy
from sklearn.feature_extraction.text import CountVectorizer, VectorizerMixin
from sklearn.base import TransformerMixin, BaseEstimator
from scipy.sparse import csr_matrix

# WordCloud
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

Using TensorFlow backend.


------
If we need to move virtual ENV to use Tensorflow we will need to install:

1. spacy

<code> conda install -c spacy spacy </code>

2. 'en_core_web_md'

<code> python -m spacy download en_core_web_md </code>

3. wordcloud

<code> conda install -c conda-forge wordcloud </code>

------

In [2]:
# import packages
data = pd.read_csv("saved_csv/df.csv")
data.drop(columns = "Unnamed: 0",inplace=True)

df = data.copy()

### Creating a model to predict comfort level using text responses
------

In [3]:
# Grabbing the responses as independent variables
corpus = df.iloc[:,-9]

# grabbing the dependent variables
dependent_class = pd.read_csv("saved_csv/q1_dependent_alt.csv")
dependent_class.drop('Unnamed: 0',axis=1,inplace=True)

dependent_class_alt = dependent_class.copy()

dependent_class_alt[dependent_class_alt < 2] = 0
dependent_class_alt[dependent_class_alt >= 2] = 1

In [4]:
# Creating a table with both independent and dependent variables
table = pd.concat([corpus,dependent_class_alt],axis=1)

# dropping columns that did not answer the question
index = table[table.iloc[:,0]=="Did not answer"].index

table.drop(index,axis=0,inplace=True)

In [5]:
# Text processing to prepare data for RNN

# Lemmatization using SpaCy
nlp = spacy.load('en_core_web_md')

sentences = []

for num in range(len(table)):
    doc = nlp(table.iloc[num,0])

    sentence = []
    for token in doc:
        sentence.append(token.lemma_)

    sentences.append(" ".join(sentence))

# Processing text with TfidfVectorizer
tf_model = TfidfVectorizer(stop_words=STOPWORDS,ngram_range=(1,3), min_df=3)
tf_vectors = tf_model.fit_transform(sentences); tf_vectors

<789x1127 sparse matrix of type '<class 'numpy.float64'>'
	with 11866 stored elements in Compressed Sparse Row format>

In [6]:
# saving files to be loaded in Hyperas functions
np.save("saved_csv/tf_vectors.npy", tf_vectors.toarray(), allow_pickle=True, fix_imports=True)

table.to_csv("saved_csv/table.csv")

In [7]:
# Tuning hyperparameter with Hyperas
# Code source: https://github.com/maxpumperla/hyperas

# for RNN

def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    tf_vectors = np.load("saved_csv/tf_vectors.npy")
    
    table = pd.read_csv("saved_csv/table.csv")
    table.drop(columns = "Unnamed: 0",inplace=True)
    
    x_train, x_test, y_train, y_test = train_test_split(tf_vectors,table.iloc[:,1].values,test_size = 0.2)

    x_train = x_train.reshape(631,1127,1)
    y_train = y_train.reshape(631,1)
    x_test = x_test.reshape(158,1127,1)
    y_test = y_test.reshape(158,1)
    
    return x_train, y_train, x_test, y_test


def create_model(x_train, y_train, x_test, y_test):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    model = Sequential()

    model.add(LSTM({{choice([32,64,96,128])}},activation={{choice(["relu","elu"])}}, 
                   input_shape = (x_train.shape[1:]), return_sequences=True, dropout={{uniform(0,1)}}))
    model.add(BatchNormalization())

    model.add(LSTM({{choice([32,64,96,128])}}, activation={{choice(["relu","elu"])}}, dropout={{uniform(0,1)}}))
    model.add(BatchNormalization())

    model.add(Dense({{choice([32,64,96,128])}}, activation={{choice(["relu","elu"])}}))
    model.add(Dropout({{uniform(0,1)}}))

    model.add(Dense(2, activation={{choice(["softmax","sigmoid"])}}))

    # setting up optimizer hyperparameters
    sgd = SGD(lr={{uniform(0,0.01)}},decay=0.0, momentum = 0.0, nesterov=False, clipnorm=2.0)

    # compile model
    model.compile(loss="sparse_categorical_crossentropy", optimizer = sgd, metrics = ["accuracy"])

    es = EarlyStopping(monitor='val_loss', mode="min", patience=2, verbose=1)

    result = model.fit(x_train,y_train, batch_size = {{choice([16, 32, 64])}}, epochs = {{choice([5, 10, 15])}}, 
                       callbacks = [es], validation_split=0.2)

    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}


if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,data=data,algo=tpe.suggest,max_evals=5,trials=Trials(),
                                          notebook_name='Capstone modelling stage v.3-RNN')
    X_train, Y_train, X_test, Y_test = data()
    print("Evaluation of best performing model:")
    print(best_model.evaluate(X_test, Y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from tensorflow.python.keras.models import Sequential
except:
    pass

try:
    from tensorflow.python.keras.utils import np_utils
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow.keras.layers import LSTM, BatchNormalization
except:
    pass

try:
    from tensorflow.keras.layers import Conv1D
except:
    pass

try:
    from tensorflow.keras.layers import MaxPooling1D
except:
    pass

try:
    from tensorflow.keras.layers import GlobalAveragePooling1D
except:
    pass

try:
    from tensorflow.keras.optimizers import SGD, Adam, RMSprop
except:
    pass

try:


 64/504 [==>...........................]           
 - ETA: 25s - loss: 0.9278 - acc: 0.4688           
                                                  
128/504 [======>.......................]           
 - ETA: 24s - loss: 0.9070 - acc: 0.4922           
                                                  
192/504 [==========>...................]           
 - ETA: 20s - loss: 0.9171 - acc: 0.5208           
                                                  
256/504 [==============>...............]           
 - ETA: 16s - loss: 0.9245 - acc: 0.5117           
                                                  
320/504 [==================>...........]           
 - ETA: 11s - loss: 0.8850 - acc: 0.5031           
                                                  
384/504 [=====================>........]           
 - ETA: 7s - loss: 0.8543 - acc: 0.5052            
                                                   
448/504 [=========================>....]           
 - ETA: 3s - loss

 - ETA: 20s - loss: 0.6926 - acc: 0.5078                                      
                                                                             
192/504 [==========>...................]                                      
 - ETA: 17s - loss: 0.6943 - acc: 0.5052                                      
                                                                             
256/504 [==============>...............]                                      
 - ETA: 13s - loss: 0.6953 - acc: 0.5078                                      
                                                                             
320/504 [==================>...........]                                      
 - ETA: 10s - loss: 0.6963 - acc: 0.4969                                      
                                                                             
384/504 [=====================>........]                                      
 - ETA: 6s - loss: 0.6954 - acc: 0.5078                 

                                                                             
320/504 [==================>...........]                                      
 - ETA: 10s - loss: 0.7027 - acc: 0.5000                                      
                                                                             
384/504 [=====================>........]                                      
 - ETA: 6s - loss: 0.7052 - acc: 0.5182                                       
                                                                              
448/504 [=========================>....]                                      
 - ETA: 3s - loss: 0.7068 - acc: 0.5045                                       
                                                                              
504/504 [==============================]                                      
 - 30s 59ms/step - loss: 0.7048 - acc: 0.4940 - val_loss: 0.6923 - val_acc: 0.5433

Epoch 8/10                                       

192/504 [==========>...................]                                      
 - ETA: 45s - loss: 0.7102 - acc: 0.5000                                      
                                                                             
224/504 [============>.................]                                      
 - ETA: 40s - loss: 0.7056 - acc: 0.5134                                      
                                                                             
256/504 [==============>...............]                                      
 - ETA: 35s - loss: 0.7022 - acc: 0.5000                                      
                                                                             
288/504 [================>.............]                                      
 - ETA: 30s - loss: 0.7039 - acc: 0.5035                                      
                                                                             
320/504 [==================>...........]                

...

480/504 [===========================>..]                                      
 - ETA: 6s - loss: 0.7068 - acc: 0.4979                                       
                                                                              
496/504 [============================>.]                                      
 - ETA: 2s - loss: 0.7072 - acc: 0.4960                                       
                                                                              
504/504 [==============================]                                      
 - 138s 274ms/step - loss: 0.7072 - acc: 0.4940 - val_loss: 0.6932 - val_acc: 0.4567

Epoch 2/10                                                                    
 16/504 [..............................]                                      
 - ETA: 2:03 - loss: 0.6298 - acc: 0.6875                                     
                                                                              
 32/504 [>.............................]     

 - 138s 273ms/step - loss: 0.7137 - acc: 0.4821 - val_loss: 0.6933 - val_acc: 0.4567

Epoch 3/10                                                                    
 16/504 [..............................]                                      
 - ETA: 1:56 - loss: 0.7068 - acc: 0.5625                                     
                                                                              
 32/504 [>.............................]                                      
 - ETA: 1:54 - loss: 0.7124 - acc: 0.4688                                     
                                                                              
 48/504 [=>............................]                                      
 - ETA: 1:50 - loss: 0.7002 - acc: 0.5000                                     
                                                                              
 64/504 [==>...........................]                                      
 - ETA: 1:48 - loss: 0.6931 - acc: 0.5156    

 - ETA: 1:00 - loss: 0.8864 - acc: 0.4375                                     
                                                                              
128/504 [======>.......................]                                      
 - ETA: 39s - loss: 0.8153 - acc: 0.4688                                      
                                                                             
192/504 [==========>...................]                                      
 - ETA: 29s - loss: 0.8093 - acc: 0.4271                                      
                                                                             
256/504 [==============>...............]                                      
 - ETA: 21s - loss: 0.7988 - acc: 0.4258                                      
                                                                             
320/504 [==================>...........]                                      
 - ETA: 15s - loss: 0.7867 - acc: 0.4344               

                                                                             
256/504 [==============>...............]                                      
 - ETA: 15s - loss: 0.7273 - acc: 0.5234                                      
                                                                             
320/504 [==================>...........]                                      
 - ETA: 11s - loss: 0.7276 - acc: 0.5062                                      
                                                                             
384/504 [=====================>........]                                      
 - ETA: 7s - loss: 0.7255 - acc: 0.5026                                       
                                                                              
448/504 [=========================>....]                                      
 - ETA: 3s - loss: 0.7213 - acc: 0.5089                                       
                                                       

384/504 [=====================>........]                                      
 - ETA: 7s - loss: 0.7429 - acc: 0.5026                                       
                                                                              
448/504 [=========================>....]                                      
 - ETA: 3s - loss: 0.7363 - acc: 0.5067                                       
                                                                              
504/504 [==============================]                                      
 - 37s 72ms/step - loss: 0.7364 - acc: 0.5060 - val_loss: 0.6899 - val_acc: 0.5433

Epoch 10/15                                                                   
 64/504 [==>...........................]                                      
 - ETA: 35s - loss: 0.6817 - acc: 0.5469                                      
                                                                             
128/504 [======>.......................]        

In [8]:
# Using results from Hyperas to create the model

def RNN_model(X_train,y_train):
    model = Sequential()

    model.add(LSTM(32,activation="elu", input_shape = (X_train.shape[1:]), return_sequences=True, dropout=0.3207527760045966))
    model.add(BatchNormalization())

    model.add(LSTM(96, activation="elu", dropout=0.7342146978592597))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='elu'))
    model.add(Dropout(0.692539034315719))

    model.add(Dense(2, activation="softmax"))

    # setting up SGD (optimizer) hyperparameters
    sgd = SGD(lr=0.004371162594318422,decay=0.0, momentum = 0.0, nesterov=False, clipnorm=2.0)

    # compile model
    model.compile(loss="sparse_categorical_crossentropy", optimizer = sgd, metrics = ["accuracy"])

    es = EarlyStopping(monitor='val_loss', mode='min', patience=2, verbose=1)

    result = model.fit(X_train,y_train, batch_size = 64, epochs = 5, callbacks = [es], validation_split=0.2)
    
    return model, result

In [9]:
X_train, X_test, y_train, y_test = train_test_split(tf_vectors,table.iloc[:,1].values,test_size = 0.2)

X_train = X_train.toarray().reshape(631,1127,1)
y_train = y_train.reshape(631,1)
X_test = X_test.toarray().reshape(158,1127,1)
y_test = y_test.reshape(158,1)

model, result = RNN_model(X_train,y_train)

Train on 504 samples, validate on 127 samples
Epoch 1/5
504/504 [==============================] - 35s 70ms/step - loss: 0.8276 - acc: 0.4861 - val_loss: 0.6924 - val_acc: 0.5906
Epoch 2/5
504/504 [==============================] - 33s 66ms/step - loss: 0.7643 - acc: 0.5179 - val_loss: 0.6926 - val_acc: 0.5906
Epoch 3/5
504/504 [==============================] - 31s 62ms/step - loss: 0.7522 - acc: 0.5357 - val_loss: 0.6918 - val_acc: 0.5906
Epoch 4/5
504/504 [==============================] - 33s 66ms/step - loss: 0.7803 - acc: 0.4901 - val_loss: 0.6914 - val_acc: 0.5906
Epoch 5/5
504/504 [==============================] - 32s 64ms/step - loss: 0.8305 - acc: 0.4980 - val_loss: 0.6915 - val_acc: 0.5906


In [10]:
# checking test accuracy
_, test_acc = model.evaluate(X_test, y_test, verbose=0)
test_acc

0.5949367073517812

In [11]:
# for CNN
def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    tf_vectors = np.load("saved_csv/tf_vectors.npy")
    
    table = pd.read_csv("saved_csv/table.csv")
    table.drop(columns = "Unnamed: 0",inplace=True)
    
    x_train, x_test, y_train, y_test = train_test_split(tf_vectors,table.iloc[:,1].values,test_size = 0.2)

    x_train = x_train.reshape(631,1127,1)
    y_train = y_train.reshape(631,1)
    x_test = x_test.reshape(158,1127,1)
    y_test = y_test.reshape(158,1)
    
    return x_train, y_train, x_test, y_test


def create_model(x_train, y_train, x_test, y_test):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    model = Sequential()

    model.add(Conv1D({{choice([32,64,96,128])}},{{choice([5,10,15,20])}},activation={{choice(["relu","elu"])}}, 
                   input_shape = (x_train.shape[1:])))
    model.add(Conv1D({{choice([32,64,96,128])}},{{choice([5,10,15,20])}},activation={{choice(["relu","elu"])}}))
    model.add(MaxPooling1D({{choice([1,2,3,4,5,6])}}))

    model.add(Conv1D({{choice([32,64,96,128])}},{{choice([5,10,15,20])}},activation={{choice(["relu","elu"])}}))
    model.add(Conv1D({{choice([32,64,96,128])}},{{choice([5,10,15,20])}},activation={{choice(["relu","elu"])}}))
    model.add(GlobalAveragePooling1D())
              
    model.add(Flatten())
    model.add(Dense({{choice([32,64,96,128])}},activation={{choice(["relu","elu"])}}))
    model.add(Dropout({{uniform(0,1)}}))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr={{uniform(0,0.01)}},decay=0.0, momentum = 0.0, nesterov=False, clipnorm=2.0)
              
    model.compile(loss="sparse_categorical_crossentropy",optimizer=sgd, metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, mode='min', patience=2, verbose=1)

    result = model.fit(x_train,y_train, batch_size = {{choice([16, 32, 64])}}, epochs = {{choice([5, 10, 15])}}, 
                       validation_split = 0.2, callbacks=[early_stop])

    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}


if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,data=data,algo=tpe.suggest,max_evals=5,trials=Trials(),
                                          notebook_name='Capstone modelling stage v.3-RNN')
    X_train, Y_train, X_test, Y_test = data()
    print("Evaluation of best performing model:")
    print(best_model.evaluate(X_test, Y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from tensorflow.python.keras.models import Sequential
except:
    pass

try:
    from tensorflow.python.keras.utils import np_utils
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow.keras.layers import LSTM, BatchNormalization
except:
    pass

try:
    from tensorflow.keras.layers import Conv1D
except:
    pass

try:
    from tensorflow.keras.layers import MaxPooling1D
except:
    pass

try:
    from tensorflow.keras.layers import GlobalAveragePooling1D
except:
    pass

try:
    from tensorflow.keras.optimizers import SGD, Adam, RMSprop
except:
    pass

try:


                                                   
384/504 [=====================>........]           
 - ETA: 1s - loss: 0.6926 - acc: 0.5339            
                                                   
448/504 [=========================>....]           
 - ETA: 0s - loss: 0.6924 - acc: 0.5379            
                                                   
504/504 [==============================]           
 - 8s 16ms/step - loss: 0.6927 - acc: 0.5278 - val_loss: 0.6936 - val_acc: 0.4882

Epoch 3/5                                          
 64/504 [==>...........................]           
 - ETA: 6s - loss: 0.6954 - acc: 0.4219            
                                                   
128/504 [======>.......................]           
 - ETA: 5s - loss: 0.6934 - acc: 0.5000            
                                                   
192/504 [==========>...................]           
 - ETA: 4s - loss: 0.6929 - acc: 0.5156            
                                 

                                                                              
432/504 [========================>.....]                                      
 - ETA: 1s - loss: 0.6932 - acc: 0.4954                                       
                                                                              
448/504 [=========================>....]                                      
 - ETA: 1s - loss: 0.6933 - acc: 0.4888                                       
                                                                              
464/504 [==========================>...]                                      
 - ETA: 0s - loss: 0.6933 - acc: 0.4957                                       
                                                                              
480/504 [===========================>..]                                      
 - ETA: 0s - loss: 0.6932 - acc: 0.5021                                       
                                                    

464/504 [==========================>...]                                      
 - ETA: 0s - loss: 0.6927 - acc: 0.5280                                       
                                                                              
480/504 [===========================>..]                                      
 - ETA: 0s - loss: 0.6927 - acc: 0.5271                                       
                                                                              
496/504 [============================>.]                                      
 - ETA: 0s - loss: 0.6927 - acc: 0.5262                                       
                                                                              
504/504 [==============================]                                      
 - 10s 19ms/step - loss: 0.6927 - acc: 0.5278 - val_loss: 0.6935 - val_acc: 0.4882

Epoch 3/10                                                                    
 16/504 [..............................]       

 - ETA: 0s - loss: 0.6925 - acc: 0.5262                                       
                                                                              
504/504 [==============================]                                      
 - 10s 20ms/step - loss: 0.6925 - acc: 0.5278 - val_loss: 0.6937 - val_acc: 0.4882

Epoch 00003: early stopping                                                   
Best validation acc of epoch:                                                 
0.48818897755127255                                                           
Train on 504 samples, validate on 127 samples                                 
Epoch 1/15                                                                    
 16/504 [..............................]                                      
 - ETA: 1:14 - loss: 0.6927 - acc: 0.2500                                     
                                                                              
 32/504 [>.............................]       

 - 14s 28ms/step - loss: 0.6935 - acc: 0.5040 - val_loss: 0.6936 - val_acc: 0.4882

Epoch 2/15                                                                    
 16/504 [..............................]                                      
 - ETA: 8s - loss: 0.6912 - acc: 0.6250                                       
                                                                              
 32/504 [>.............................]                                      
 - ETA: 8s - loss: 0.6914 - acc: 0.5625                                       
                                                                              
 48/504 [=>............................]                                      
 - ETA: 8s - loss: 0.6881 - acc: 0.6250                                       
                                                                              
 64/504 [==>...........................]                                      
 - ETA: 8s - loss: 0.6867 - acc: 0.6406        

                                                                              
 48/504 [=>............................]                                      
 - ETA: 9s - loss: 0.6941 - acc: 0.4792                                       
                                                                              
 64/504 [==>...........................]                                      
 - ETA: 8s - loss: 0.6950 - acc: 0.4688                                       
                                                                              
 80/504 [===>..........................]                                      
 - ETA: 9s - loss: 0.6927 - acc: 0.5000                                       
                                                                              
 96/504 [====>.........................]                                      
 - ETA: 8s - loss: 0.6901 - acc: 0.5312                                       
                                                    

                                                                             
128/504 [======>.......................]                                      
 - ETA: 22s - loss: 0.6935 - acc: 0.5000                                      
                                                                             
160/504 [========>.....................]                                      
 - ETA: 19s - loss: 0.6933 - acc: 0.5250                                      
                                                                             
192/504 [==========>...................]                                      
 - ETA: 17s - loss: 0.6932 - acc: 0.5417                                      
                                                                             
224/504 [============>.................]                                      
 - ETA: 14s - loss: 0.6933 - acc: 0.5268                                      
                                                        

192/504 [==========>...................]                                      
 - ETA: 12s - loss: 0.6946 - acc: 0.5260                                      
                                                                             
224/504 [============>.................]                                      
 - ETA: 11s - loss: 0.6947 - acc: 0.5312                                      
                                                                             
256/504 [==============>...............]                                      
 - ETA: 10s - loss: 0.6942 - acc: 0.5234                                      
                                                                             
288/504 [================>.............]                                      
 - ETA: 8s - loss: 0.6947 - acc: 0.5312                                       
                                                                              
320/504 [==================>...........]               

 - ETA: 2s - loss: 0.6928 - acc: 0.5268                                      
                                                                             
504/504 [==============================]                                     
 - 20s 40ms/step - loss: 0.6927 - acc: 0.5278 - val_loss: 0.6943 - val_acc: 0.4882

Epoch 00003: early stopping                                                  
Best validation acc of epoch:                                                
0.48818897848992837                                                          
100%|██████████| 5/5 [03:46<00:00, 47.98s/it, best loss: -0.5275590574647498]
Evaluation of best performing model:
158/158 [==============================] - 2s 11ms/step
[0.6931702494621277, 0.46202531570120703]
Best performing model chosen hyper-parameters:
{'Conv1D': 2, 'Conv1D_1': 1, 'Conv1D_2': 1, 'Conv1D_3': 3, 'Conv1D_4': 1, 'Conv1D_5': 3, 'Conv1D_6': 1, 'Conv1D_7': 1, 'Conv1D_8': 2, 'Dropout': 0.9975531408239052, 'MaxPooling1D': 0, 'act

In [ ]:
# Using results from Hyperas to create the model
def CNN_model(x_train,y_train):
    CNN_model = Sequential()

    CNN_model.add(Conv1D(96,10,activation="relu",input_shape = (x_train.shape[1:])))
    CNN_model.add(Conv1D(64,20,activation="relu"))
    CNN_model.add(MaxPooling1D(0))

    CNN_model.add(Conv1D(64,20,activation="elu"))
    CNN_model.add(Conv1D(64,10,activation="relu"))
    CNN_model.add(GlobalAveragePooling1D())

    CNN_model.add(Flatten())
    CNN_model.add(Dense(96,activation="elu"))
    CNN_model.add(Dropout(0.9912013870496312))
    CNN_model.add(Dense(10, activation='softmax'))

    sgd = SGD(lr=0.00026079803111884515,decay=0.0, momentum = 0.0, nesterov=False, clipnorm=2.0)

    CNN_model.compile(loss="sparse_categorical_crossentropy",optimizer=sgd, metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, mode='min', patience=2, verbose=1)

    results = CNN_model.fit(x_train,y_train, batch_size=32, epochs=10, validation_split = 0.2, callbacks=[early_stop])
    
    return CNN_model, results

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(tf_vectors,table.iloc[:,1].values,test_size = 0.2)

x_train = x_train.toarray().reshape(631,1127,1)
y_train = y_train.reshape(631,1)
x_test = x_test.toarray().reshape(158,1127,1)
y_test = y_test.reshape(158,1)

CNN_model, results = CNN_model(x_train,y_train)

In [ ]:
# checking test accuracy
_, test_acc = CNN_model.evaluate(x_test, y_test, verbose=0)
test_acc